In [ ]:
import pandas as pd
import scipy.stats as stats
from sklearn import datasets
from scipy.stats import shapiro, levene
from statsmodels.stats.multicomp import pairwise_tukeyhsd


# Carregar o dataset Iris
iris = datasets.load_iris()
df = pd.DataFrame(iris.data, columns=iris.feature_names)
df['species'] = iris.target

# Normalização dos dados (Z-score)
df['sepal width (cm)'] = (df['sepal width (cm)'] - df['sepal width (cm)'].mean()) / df['sepal width (cm)'].std()

# Exibir as hipóteses
print("Teste ANOVA para a largura da sépala no conjunto de dados Iris\n")
print("Hipótese Nula (H0): Não há diferença significativa entre as médias da largura da sépala entre as espécies.")
print("Hipótese Alternativa (HA): Pelo menos uma das médias da largura da sépala é significativamente diferente entre as espécies.\n")

# Separar os dados por espécie
species_groups = [df[df['species'] == i]['sepal width (cm)'] for i in range(3)]

# Teste de Normalidade (Shapiro-Wilk)
print("Teste de Normalidade (Shapiro-Wilk) para cada grupo:")
for i in range(3):
    stat, p = shapiro(species_groups[i])
    print(f"  Espécie {i}: p-Value = {p:.5f}")
print()

# Teste de Homogeneidade das Variâncias (Levene)
stat, p = levene(*species_groups)
print(f"Teste de Homogeneidade de Variâncias (Levene): p-Value = {p:.5f}")
if p < 0.05:
    print("  -> As variâncias são diferentes entre os grupos. Isso pode impactar a ANOVA.\n")
else:
    print("  -> As variâncias são homogêneas, condição ideal para ANOVA.\n")

# Realizar ANOVA apenas para largura da sépala
print("Resultados do Teste ANOVA para a largura da sépala:\n")
f_stat, p_value = stats.f_oneway(*species_groups)
print(f"Largura da sépala:")
print(f"  F-Statistic: {f_stat:.2f}")
print(f"  p-Value: {p_value:.5f}")
if p_value < 0.05:
    print("  -> Rejeitamos H0: Existe diferença significativa entre as médias das espécies.\n")

    # Teste post-hoc (Tukey HSD)
    print("Teste de Comparação Múltipla (Tukey HSD) para identificar diferenças entre grupos:\n")
    tukey = pairwise_tukeyhsd(df['sepal width (cm)'], df['species'])
    print(tukey)
else:
    print("  -> Não rejeitamos H0: Não há evidências suficientes para afirmar que as médias são diferentes.\n")

# Executar a análise
# comclusão essa basse de dados não e recomendado que se utilise o teste anova pois os dados não seguem uma distribuição
# normal e não tem Homogeneidade o que esplica


Teste ANOVA para a largura da sépala no conjunto de dados Iris

Hipótese Nula (H0): Não há diferença significativa entre as médias da largura da sépala entre as espécies.
Hipótese Alternativa (HA): Pelo menos uma das médias da largura da sépala é significativamente diferente entre as espécies.

Teste de Normalidade (Shapiro-Wilk) para cada grupo:
  Espécie 0: p-Value = 0.27153
  Espécie 1: p-Value = 0.33800
  Espécie 2: p-Value = 0.18090

Teste de Homogeneidade de Variâncias (Levene): p-Value = 0.55552
  -> As variâncias são homogêneas, condição ideal para ANOVA.

Resultados do Teste ANOVA para a largura da sépala:

Largura da sépala:
  F-Statistic: 49.16
  p-Value: 0.00000
  -> Rejeitamos H0: Existe diferença significativa entre as médias das espécies.

Teste de Comparação Múltipla (Tukey HSD) para identificar diferenças entre grupos:

Multiple Comparison of Means - Tukey HSD, FWER=0.05 
group1 group2 meandiff p-adj   lower   upper  reject
---------------------------------------------


Nenhum grupo ou grupos insuficientes atendem à condição de normalidade para aplicar ANOVA.


In [4]:
import pandas as pd
import scipy.stats as stats

# Carregar o dataset
url = "https://archive.ics.uci.edu/ml/machine-learning-databases/wine-quality/winequality-red.csv"
df = pd.read_csv(url, sep=';')

# Definir os limites para categorizar o teor alcoólico
low_threshold = df['alcohol'].quantile(1/3)
high_threshold = df['alcohol'].quantile(2/3)

def categorize_alcohol(value):
    if value <= low_threshold:
        return 'Baixo'
    elif value <= high_threshold:
        return 'Médio'
    else:
        return 'Alto'

# Aplicar a categorização
df['Alcohol_Category'] = df['alcohol'].apply(categorize_alcohol)

# Separar os grupos
group_low = df[df['Alcohol_Category'] == 'Baixo']['quality']
group_medium = df[df['Alcohol_Category'] == 'Médio']['quality']
group_high = df[df['Alcohol_Category'] == 'Alto']['quality']

print(f"Tamanho das amostras:")
print(f"Baixo: {len(group_low)} vinhos")
print(f"Médio: {len(group_medium)} vinhos")
print(f"Alto: {len(group_high)} vinhos")
# Teste de Normalidade (Shapiro-Wilk)

print("Teste de Normalidade (Shapiro-Wilk) para cada grupo:")
print(f"Baixo: p-valor = {stats.shapiro(group_low).pvalue:.4f}")
print(f"Médio: p-valor = {stats.shapiro(group_medium).pvalue:.4f}")
print(f"Alto: p-valor = {stats.shapiro(group_high).pvalue:.4f}")

# Teste de Homogeneidade de Variâncias (Levene)
levene_test = stats.levene(group_low, group_medium, group_high)
print("\nTeste de Homogeneidade de Variâncias (Levene):")
print(f"p-valor = {levene_test.pvalue:.4f}")

# Aplicar o teste ANOVA
anova_result = stats.f_oneway(group_low, group_medium, group_high)

# Exibir os resultados
print("\nHipóteses do Teste ANOVA:")
print("H0: As médias de qualidade dos vinhos nos diferentes grupos de teor alcoólico são iguais.")
print("H1: Pelo menos um dos grupos tem média de qualidade diferente.")

print("\nResultados do Teste ANOVA:")
print(f"F-Estatística: {anova_result.statistic:.4f}, p-valor: {anova_result.pvalue:.4f}")

# Interpretar o resultado
a = 0.05
if anova_result.pvalue < a:
    print("Rejeitamos a hipótese nula: há diferença significativa entre as médias de qualidade dos grupos.")
else:
    print("Não rejeitamos a hipótese nula: não há evidências suficientes para afirmar que há diferença entre as médias de qualidade dos grupos.")


Tamanho das amostras:
Baixo: 552 vinhos
Médio: 531 vinhos
Alto: 516 vinhos
Teste de Normalidade (Shapiro-Wilk) para cada grupo:
Baixo: p-valor = 0.0000
Médio: p-valor = 0.0000
Alto: p-valor = 0.0000

Teste de Homogeneidade de Variâncias (Levene):
p-valor = 0.0000

Hipóteses do Teste ANOVA:
H0: As médias de qualidade dos vinhos nos diferentes grupos de teor alcoólico são iguais.
H1: Pelo menos um dos grupos tem média de qualidade diferente.

Resultados do Teste ANOVA:
F-Estatística: 181.0073, p-valor: 0.0000
Rejeitamos a hipótese nula: há diferença significativa entre as médias de qualidade dos grupos.
